# imports

In [1]:
import pandas as pd

## constants

In [2]:
NUMBER_OF_ELEMENTS_TO_TAKE = 200

# Preprocess dataframe

## Load data and concat two years

In [3]:
data_2018 = pd.read_csv("data2018.csv", encoding="cp1251")
data_2019 = pd.read_csv("data2019.csv", encoding="cp1251")
df = pd.concat([data_2018, data_2019])
df = df.sample(1100000, random_state=2).reset_index(drop=True)
df

,Рік,Область,Учбовий заклад,Спеціальність,Факультет,Місце у рейтингу,Пріоритет,Конкурсний бал,Статус заяви,Інфо про спеціальність,Квоти
0,2019,м. Київ,Київський національний торговельно-економічний...,081 Право,Факультет міжнародної торгівлі та права,754,К,155.900,Допущено,Українська мова та література (ЗНО): 154\r\nІс...,—
1,2018,Львівська область,Львівський національний університет імені Іван...,035 Філологія,Філологічний факультет,390,К,165.852,Допущено,Середній бал документа про освіту: 10.6\r\nУкр...,—
2,2019,Львівська область,Львівський національний університет імені Іван...,"291 Міжнародні відносини, суспільні комунікаці...",Факультет міжнародних відносин,480,4,187.068,Допущено,—,—
3,2018,м. Київ,Київський національний лінгвістичний університет,081 Право,Факультет економіки і права,26,6,193.850,Рекомендовано (контракт),Українська мова та література (ЗНО): 193\r\nІс...,—
4,2019,м. Київ,Національний університет харчових технологій,073 Менеджмент,Навчально-науковий інститут економіки і управл...,162,К,119.400,Рекомендовано (к),Українська мова та література (ЗНО): 131\r\nМа...,—
...,...,...,...,...,...,...,...,...,...,...,...
1099995,2018,Львівська область,Львівський національний університет імені Іван...,081 Право,Юридичний факультет,606,5,192.780,Допущено,Українська мова та література (ЗНО): 191\r\nІс...,—
1099996,2018,Львівська область,Львівський національний університет імені Іван...,061 Журналістика,Факультет журналістики,949,4,169.065,Допущено,Українська мова та література (ЗНО): 173\r\nІс...,—
1099997,2019,Івано-Франківська область,Прикарпатський національний університет імені ...,241 Готельно-ресторанна справа,Факультет туризму,89,6,159.171,Допущено,Українська мова та література (ЗНО): 164\r\nІн...,—
1099998,2018,м. Київ,Державний вищий навчальний заклад «Київський н...,051 Економіка,Факультет мiжнародної економiки i менеджменту,243,2,195.200,Рекомендовано (контракт),Українська мова та література (ЗНО): 196\r\nМа...,—


## Rename, remove and clean columns
0 - didn't pass at all

1 - pass to contract

2 - pass to budget

In [4]:
df.columns = [
    "year",
    "district",
    "university",
    "specialty",
    "faculty",
    "rating",
    "priority",
    "points",
    "status",
    "info",
    "quotas"
]
df = df.drop(["year", "faculty", "info", "quotas", "rating", "priority"], axis=1)
df["status"] = df["status"].map({
    'До наказу (контракт)': 1,
    'Рекомендовано (контракт)': 1,
    'Відмова': 0,
    'Скасовано НЗ': 0,
    'Скасовано (втрата пріор.)': 0,
    'Зареєстровано': 0,
    'Затримано': 0,
    'Відраховано': 0,
    'До наказу (бюджет)': 2,
    'Відхилено (бюджет)': 0,
    'Допущено': 0,
    'Рекомендовано (бюджет)': 2,
    'Заява надійшла з сайту': 0,
    'Відхилено (контракт)': 0,
    'Допущено (контракт, за ріш. ПК)': 1,
    'Рекомендовано (к)': 1,
    'До наказу (к)': 1,
    'До наказу (б)': 2,
    'Відхилено (б)': 0,
    'Рекомендовано (б)': 2,
    'Відхилено (к)': 0,
    'Допущено (к, за ріш. ПК)': 1
})
df["specialty"] = df["specialty"].map(lambda val: val[:3])

In [5]:
df = df.query("points <= 200 and points >= 100")
districts = df.district.unique()
specialties = df.specialty.unique()
universities = df.university.unique()
accum = []
for dis in districts:
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["district"] == dis) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
for spec in specialties:
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
for un in universities:
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
df = pd.concat(accum)
df

,district,university,specialty,points,status
0,м. Київ,Київський національний торговельно-економічний...,081,155.900,0
8,м. Київ,Київський національний університет культури і ...,034,164.300,0
9,м. Київ,Київський університет імені Бориса Грінченка,035,167.700,0
13,м. Київ,Київський національний університет імені Тарас...,113,190.000,0
15,м. Київ,Київський національний університет культури і ...,027,151.500,0
...,...,...,...,...,...
1039510,Черкаська область,Уманська філія Приватного вищого навчального з...,073,140.454,1
875320,Дніпропетровська область,Кам`янський медичний коледж,223,183.000,0
907039,Івано-Франківська область,Надвірнянський коледж Національного транспортн...,029,127.000,0
919912,Донецька область,Вугледарський коледж Донецького державного уні...,072,132.000,0


In [6]:
budget = df.query("status == 2")
contract = df.query("status == 1").sample(n=5000)
not_passed = df.query("status == 0").sample(n=2000)
df = pd.concat([budget, contract, not_passed])
df

,district,university,specialty,points,status
126,м. Київ,Національна академія образотворчого мистецтва ...,191,181.500,2
183,м. Київ,Національний університет харчових технологій,181,170.000,2
198,м. Київ,Національний технічний університет України «Ки...,131,182.172,2
296,м. Київ,Київський національний університет культури і ...,061,186.125,2
370,м. Київ,Київський національний університет імені Тарас...,101,173.655,2
...,...,...,...,...,...
210192,Хмельницька область,Хмельницька гуманітарно-педагогічна академія,242,156.000,0
39466,м. Київ,Київський національний університет імені Тарас...,124,167.750,0
23155,Черкаська область,Черкаський національний університет імені Богд...,017,138.491,0
108430,м. Київ,Національний технічний університет України «Ки...,134,124.400,0


In [7]:
df.status = df.status.dropna()
df.head()
df

,district,university,specialty,points,status
126,м. Київ,Національна академія образотворчого мистецтва ...,191,181.500,2
183,м. Київ,Національний університет харчових технологій,181,170.000,2
198,м. Київ,Національний технічний університет України «Ки...,131,182.172,2
296,м. Київ,Київський національний університет культури і ...,061,186.125,2
370,м. Київ,Київський національний університет імені Тарас...,101,173.655,2
...,...,...,...,...,...
210192,Хмельницька область,Хмельницька гуманітарно-педагогічна академія,242,156.000,0
39466,м. Київ,Київський національний університет імені Тарас...,124,167.750,0
23155,Черкаська область,Черкаський національний університет імені Богд...,017,138.491,0
108430,м. Київ,Національний технічний університет України «Ки...,134,124.400,0


## Get values for frontend

In [8]:
df.district.unique()

array(['м. Київ', 'Львівська область', 'Миколаївська область',
       'Запорізька область', 'Харківська область', 'Вінницька область',
       'Рівненська область', 'Одеська область', 'Кіровоградська область',
       'Херсонська область', 'Чернівецька область',
       'Чернігівська область', 'Івано-Франківська область',
       'Дніпропетровська область', 'Черкаська область',
       'Хмельницька область', 'Луганська область',
       'Тернопільська область', 'Сумська область', 'Київська область',
       'Житомирська область', 'Волинська область', 'Донецька область',
       'Полтавська область', 'Закарпатська область'], dtype=object)

In [9]:
df.specialty.unique()

array(['191', '181', '131', '061', '101', '192', '133', '034', '053',
       '201', '162', '125', '035', '072', '172', '152', '123', '124',
       '052', '029', '113', '025', '051', '111', '122', '193', '106',
       '272', '024', '023', '151', '026', '017', '242', '033', '141',
       '202', '173', '032', '231', '207', '121', '016', '014', '161',
       '021', '204', '292', '105', '227', '171', '081', '281', '126',
       '091', '241', '275', '076', '163', '028', '291', '073', '271',
       '142', '075', '022', '134', '208', '015', '103', '182', '132',
       '071', '273', '104', '013', '012', '102', '186', '261', '205',
       '153', '187', '274', '112', '194', '143', '054', '232', '135',
       '223', '144', '183', '145', '136', '293', '185', '206', '226',
       '184', '263', '224', '262', '211', '229', '203', '027', '031',
       '041', '251', '256', '011'], dtype=object)

In [14]:
df.university.unique()

array(['Національна академія образотворчого мистецтва і архітектури',
       'Національний університет харчових технологій',
       'Національний технічний університет України «Київський політехнічний інститут імені Ігоря Сікорського»',
       'Київський національний університет культури і мистецтв',
       'Київський національний університет імені Тараса Шевченка',
       'Київський національний університет будівництва і архітектури',
       'Київський національний університет технологій та дизайну',
       'Національний університет біоресурсів і природокористування України',
       'Національний авіаційний університет',
       'Київський національний торговельно-економічний університет',
       'Таврійський національний університет імені В.І.Вернадського',
       'Національна музична академія України імені П.І. Чайковського',
       'Національний педагогічний університет імені М.П. Драгоманова',
       'Київський державний інститут декоративно-прикладного мистецтва і дизайну імені Ми

## Split into two datasets

- The first defines people who didn't pass to contract and didn't pass at all as the same people. This model is used to check people who wants to apply for budget
- The second defines people who pass to contract and to budget as the same category, since if you pass to budget you'll pass to contract

In [10]:
df_budget = df.copy()
df_contract = df.copy()
df_budget["status"] = df_budget["status"].map({
    1: -1,
    2: 1,
    0: -1
})
df_contract["status"] = df_contract["status"].map({
    1: 1,
    2: 1,
    0: -1
})


In [11]:
df_budget.head()

,district,university,specialty,points,status
126,м. Київ,Національна академія образотворчого мистецтва ...,191,181.500,1
183,м. Київ,Національний університет харчових технологій,181,170.000,1
198,м. Київ,Національний технічний університет України «Ки...,131,182.172,1
296,м. Київ,Київський національний університет культури і ...,061,186.125,1
370,м. Київ,Київський національний університет імені Тарас...,101,173.655,1


In [12]:
df_contract.head()

,district,university,specialty,points,status
126,м. Київ,Національна академія образотворчого мистецтва ...,191,181.500,1
183,м. Київ,Національний університет харчових технологій,181,170.000,1
198,м. Київ,Національний технічний університет України «Ки...,131,182.172,1
296,м. Київ,Київський національний університет культури і ...,061,186.125,1
370,м. Київ,Київський національний університет імені Тарас...,101,173.655,1


## Dump cleaned_data set

In [13]:
df_budget.to_csv("budget.csv", index=False)
df_contract.to_csv("contract.csv", index=False)